In [2]:
import shutil
import os
RESEARCH_WORK_PATH = "/content/drive/MyDrive/Colab Notebooks/BachelorResearch/"


data_dir = "/content/datas/"
if os.path.exists(data_dir):
  shutil.rmtree(data_dir)
for dirs in os.listdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/5s_0.5shift/wav/"):
  if "Q" in dirs:
    shutil.copytree(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/5s_0.5shift/wav/" + dirs, data_dir + dirs)

In [3]:
import librosa
import numpy as np
import torch
import pickle
from tqdm import tqdm

scaler = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/BachelorResearch/models/1d_scaler_5s_0.5shift.pickle', "rb"))

def min_max(x): # do min_max normalization ?->https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html
  min = 0
  max = 159.42822
  x = np.abs(x)
  result = (x-min)/(max-min)
  return result

def feature(song_path):
  y, sr = librosa.load(song_path)
  
  n_fft = 512
  hop_length = 256
  stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
  stft = min_max(stft)
  stft = np.expand_dims(stft, axis=2)
  stft = torch.from_numpy(stft)
  stft = stft.permute(2,1,0)


  chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
  # rmse = librosa.feature.rmse(y=y)
  rmse = librosa.feature.rms(y=y) # librosaのバージョンが0.7.0以上の場合はこちらを実行
  spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  zcr = librosa.feature.zero_crossing_rate(y)
  mfcc = librosa.feature.mfcc(y=y, sr=sr)
  f1d = [np.mean(chroma_stft), np.mean(rmse), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]    
  for e in mfcc:
    f1d.append(np.mean(e))
  f1d = np.array(f1d, dtype=float)
  f1d = np.expand_dims(f1d,0)
  f1d = scaler.transform(f1d)
  f1d = np.squeeze(f1d)
  f1d = torch.from_numpy(f1d).float()


  label = int(song_path.split("/")[-1].split(".")[-3][1])-1
  return [f1d, stft], label

def path_to_audiofiles(dir_folder):
  list_of_audio = []
  for file in os.listdir(dir_folder):
    if file.endswith(".wav"):
      directory = "%s%s" % (dir_folder, file)
      list_of_audio.append(directory)
  return list_of_audio


dst_dir = "/content/dst/"
if not os.path.exists(dst_dir):
  os.mkdir(dst_dir)
  os.mkdir(dst_dir+"Q1")
  os.mkdir(dst_dir+"Q2")
  os.mkdir(dst_dir+"Q3")
  os.mkdir(dst_dir+"Q4")

Qs = ['Q1', 'Q2', 'Q3', 'Q4']

for q in Qs:
  for a in tqdm(path_to_audiofiles(data_dir + q + "/")):
    x,y = feature(a)
    with open(dst_dir + q + '/' + a.split("/")[-1] + '.pickle', mode='wb') as f:
      pickle.dump([x,y], f)


100%|██████████| 9200/9200 [20:44<00:00,  7.39it/s]


In [4]:
for q in "Q1 Q2 Q3 Q4".split(" "):
  d = "/content/datas/" + q + "/"
  print(q, sum(os.path.isfile(os.path.join(d, name)) for name in os.listdir(d)))

Q1 9200
Q2 9200
Q3 9200
Q4 9200


In [5]:
shutil.copytree(dst_dir, RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/5s_0.5shift/pickles1/")

'/content/drive/MyDrive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/5s_0.5shift/pickles1/'

In [6]:
for q in "Q1 Q2 Q3 Q4".split(" "):
  d = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/5s_/pickles1/" + q + "/"
  print(q, sum(os.path.isfile(os.path.join(d, name)) for name in os.listdir(d)))

Q1 2800
Q2 2800
Q3 2800
Q4 2800
